# Package Development

In the last week we've looked a bit at Julia's package ecosystem, and practiced using git repositories.  Homework 3 included contributing to someone else's project by creating a pull request on the course GitHub repository, so you now have the tools you need to contribute to someone else's package if you have the interest.

This is great if you fix a bug in someone else's code,or get involved in an existing package, but what if:

1. You want functionality to exist in Julia that no one has worked on yet?
2. You have a reasearch project or course project you want to let other people play with?

Today we're going to talk about how you can make your own package, and have it work for anyone who wants to use it right out of the box.

Julia has [documentation](http://julia.readthedocs.org/en/latest/manual/packages/) on how to make a package.  If you want to make a package to be included in the Julia package registry, refer there for naming conventions and further information.  We'll take a look at the basic workflow and examine what different files are doing.

## Generating a Package

You can generate the basic files for your package by using Pkg.generate("name", "license")

In [ ]:
#Pkg.generate("cme257demo", "MIT")

This will initialize a new package in your packages directory, which you can begin to edit.  Here's what I see:

```bash
julia> Pkg.generate("cme257demo", "MIT")
INFO: Initializing cme257demo repo: /home/brad/.julia/v0.4/cme257demo
INFO: Generating LICENSE.md
INFO: Generating README.md
INFO: Generating src/cme257demo.jl
INFO: Generating test/runtests.jl
INFO: Generating REQUIRE
INFO: Generating .travis.yml
INFO: Generating appveyor.yml
INFO: Generating .gitignore
INFO: Committing cme257demo generated files
```


* LICENSE.md contains the license that you specified
* README.md will contain information that people first see when the find your package on GitHub so make it nice and informative.
* REQUIRE contains package dependency information.
* .travis.yml and appveyor.yml contain information for the Travis and Appveyor CI platforms to let them know how to test your package.
* src/cme257demo.jl starts a cme257demo module for you to put code. This will be loaded when you use a variant of
```julia
using cme257demo
```
* test/runtests.jl starts a test file for you to put your tests.  These are run with
```julia
Pkg.test("cme257demo")
```

# Exercize 1

* use Pkg.generate to generate your own package.  
* add a hello() function to print a hello message
* verify that you can print the message from julia after loading your package.
* How can you make it so you can call your function directly? (not using importall)
```julia
# this:
hello()
# instead of 
cme257demo.hello()
```
* add a demofun() function that will add two integers
* (if you have time) add a function that returns a random draw from the exponential distribution ($\lambda = 1$).  Add Distributions to your REQUIRE file.

In [ ]:
importall cme257demo

In [ ]:
hello()

In [ ]:
demofun(1, 2)

In [ ]:
exprand()

## Testing packages

It is important to include tests for your package as you go along.  How else are you going to convince someone that your code works (including future you)?  Having a comprehensive test suite also alerts you when you break something when you go back and make changes to your package. Ideally, you will have tests for every function and type you create, and test edge cases as well as possible.  If you haven't ever done this before, begin by including all the little tests you do on the command line when you're making sure your code works.

Julia includes several useful macros designed for testing code.  These can be found in [Base.Test](https://github.com/JuliaLang/julia/blob/master/base/test.jl), which you need to include in your test script (the package generator automatically does this for you).

* @test - tests boolean expression, and displays a message if the expression is evaluated to be false.


In [ ]:
using Base.Test

In [ ]:
@test 1 == 1 && (print("hello") || 1)

In [ ]:
@test 1 == 2 || print("hello")

* @test_throws - test to see if a function throws a particular error when you give it bad input

In [ ]:
length(+)

In [ ]:
@test_throws MethodError length(+)
;

* You can test approximate equality using the isapporox() function or $\approx$ operator.  This is useful for floating point computations.

In [ ]:
ɛ = 2e-16
@test 1 ≈ (1 + ɛ)
@test isapprox(1, 1 + ɛ)
# \approx ≈

In [ ]:
@testset  begin
    @test 1 == 1
    @test 1 == 2
    @test 1 == 1
end

## Exercise 2

* add some tests to your demofun() function in your package. Include at least one @test and at least one @test_throws
* what is the tolerance for isapprox() to fail?  Can you change this?

In [ ]:
Pkg.test("cme257demo")

## Continuous integration

As we talked about in the last class, using continuous integration is a great way to show off that your code is working to people who are interested in your repository.

Travis CI is a continuous integration platform for Mac/Linux builds.  Appveyor is used for Windows builds in Julia.  We'll demo how to use [Travis CI](https://travis-ci.org).

* You need to set up an account for Travis CI.  If you're signed into GitHub, you can link the two.
* to set up Travis CI for a repository, you need admin access to the repository

[Demo Repository](https://github.com/bnels/cme257demo.jl)

[Travis Julia instructions](http://docs.travis-ci.com/user/languages/julia/)

## Precompiling packages

As you know, sometimes packages can take a while to load.  The way to speed things up is through pre-compilation.  There are two strategies to do this:

1. add `__precompile__()` to the top of your module file (before the module begins)
2. add your package to your compile cache
```julia
Base.compilecache("cme257demo")
```

You can also prevent your package from being precompiled by using `__precompile__(false)`.

We'll talk a bit more about pre-compiling when we talk about calling libraries, since you need to make sure you load libraries at runtime.

Juila has more information [here](http://julia.readthedocs.org/en/latest/manual/modules/#module-initialization-and-precompilation).

## Exercise 3

* modify your package to incrementally precompile
* add your package to your precompile cache

## Adding a Package to GitHub

At some point, you'll want to put your package on GitHub to share with others.  To do this, create a new GitHub repository with your account with your desired name.  Then, you'll add the remote repository as the origin of your local repository.

```bash
cd /path/to/cme257demo
git remote -v # doesn't print anything yet
git remote add origin https://github.com/user/repo.git
git remote -v # will now print the new origin
```

See also, [GitHub's help page on this](https://help.github.com/articles/adding-a-remote/)

If you've set up git on your machine to know your github user name, this may already be done for you with the Pkg.generate() command.

```bash
git config --global github.user "USERNAME"
```

You can also edit your .git/config file in the repository if Pkg.generate() did not quite get it right.

## Registering a Package in the Julia Package Registry

You'll need to submit a pull request to the [METADATA.jl repository](https://github.com/JuliaLang/METADATA.jl).

Julia encourages developers to do this via the Pkg commands:

```julia
Pkg.register("cme257demo") # updates your copy of METADATA appropriately
# don't publish unless you actually want to publish 
Pkg.publish("cme257demo") # forks you a copy of METADATA and generates a pull request to the official repository
```

Julia has more instructions [here](http://julia.readthedocs.org/en/latest/manual/packages/#tagging-and-publishing-your-package).